<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#INCEPTION" data-toc-modified-id="INCEPTION-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>INCEPTION</a></span></li></ul></div>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models 
from tensorflow import keras

## INCEPTION
<img src="../images/inception.jpg" />

GoogLeNet uses a stack of a total of 9 inception blocks and global average pooling to generate its estimates. Maximum pooling between inception blocks reduced the
dimensionality. The first part is identical to AlexNet and LeNet, the stack of blocks is inherited
from VGG and the global average pooling avoids a stack of fully-connected layers at the end. The
architecture is depicted below
<img src="../images/inception1.jpg" />
<img src="../images/inception2.jpg" />

In [3]:
class Inception_block(tf.keras.Model):
    def __init__(self,c1,c2,c3,c4,**kwargs):
        super().__init__(**kwargs)
        # Path 1 is a single 1 x 1 convolutional layer
        self.p1_1=layers.Conv2D(filters=c1,kernel_size=1,activation='relu')
        # Path 2 is a 1 x 1 convolutional layer followed by a 3 x 3
        # convolutional layer
        self.p2_1=layers.Conv2D(filters=c2[0],kernel_size=1,activation='relu')
        self.p2_2=layers.Conv2D(filters=c2[1],kernel_size=3,padding='same',activation='relu')
         # Path 2 is a 1 x 1 convolutional layer followed by a 5 x 5
        # convolutional layer
        self.p3_1=layers.Conv2D(filters=c3[0],kernel_size=1,activation='relu')
        self.p3_2=layers.Conv2D(filters=c3[1],kernel_size=5,padding='same',activation='relu')
         # Path 4 is a 3 x 3 maximum pooling layer followed by a 1 x 1
        # convolutional layer
        self.p4_1=layers.MaxPool2D(pool_size=3,padding='same',strides=1)
        self.p4_2=layers.Conv2D(filters=c4,kernel_size=1,activation='relu')
    def call(self,x):
        p1=self.p1_1(x)
        p2=self.p2_2(self.p2_1(x))
        p3=self.p3_2(self.p3_1(x))
        p4=self.p4_2(self.p4_1(x))
        return layers.Concatenate()([p1, p2, p3, p4])

In [4]:
def inception():
    return tf.keras.models.Sequential([
        layers.Conv2D(filters=64,kernel_size=7,strides=2,padding='same',activation='relu'),
        layers.MaxPool2D(pool_size=3,padding='same',strides=2),
        
        layers.Conv2D(64,kernel_size=1,activation='relu'),
        layers.Conv2D(192,kernel_size=3,padding='same',activation='relu'),
        layers.MaxPool2D(pool_size=3,padding='same',strides=2),
        # inception(3a)
        Inception_block(c1=64,c2=(96,128),c3=(16,32),c4=32),
               # inception(3b)
        Inception_block(c1=128,c2=(128,192),c3=(32,96),c4=64),
        layers.MaxPool2D(pool_size=3,padding='same',strides=2),
        # inception(4a)
        Inception_block(c1=192,c2=(96,208),c3=(16,48),c4=64),
              # inception(4b)
        Inception_block(c1=160,c2=(112,224),c3=(24,64),c4=64),
        # inception(4c)
        Inception_block(c1=128,c2=(128,256),c3=(24,64),c4=64),
        # inception(4d)
        Inception_block(112,(144,288),(32,64),64),
        # inception(4e)
        Inception_block(256,(160,320),(32,128),128),
        layers.MaxPool2D(pool_size=3,padding='same',strides=2),
         # inception(5a)
        Inception_block(256, (160, 320), (32, 128), 128),
              # inception(5b)
        Inception_block(384, (192, 384), (48, 128), 128),
        layers.MaxPool2D(pool_size=3,padding='same',strides=2),
        layers.GlobalAvgPool2D(),
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(10,activation='softmax')
    ])

In [5]:
X = tf.random.uniform(shape=(1, 1, 96, 96))
for layer in inception().layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Conv2D output shape:	 (1, 1, 48, 64)
MaxPooling2D output shape:	 (1, 1, 24, 64)
Conv2D output shape:	 (1, 1, 24, 64)
Conv2D output shape:	 (1, 1, 24, 192)
MaxPooling2D output shape:	 (1, 1, 12, 192)
Inception_block output shape:	 (1, 1, 12, 256)
Inception_block output shape:	 (1, 1, 12, 480)
MaxPooling2D output shape:	 (1, 1, 6, 480)
Inception_block output shape:	 (1, 1, 6, 512)
Inception_block output shape:	 (1, 1, 6, 512)
Inception_block output shape:	 (1, 1, 6, 512)
Inception_block output shape:	 (1, 1, 6, 528)
Inception_block output shape:	 (1, 1, 6, 832)
MaxPooling2D output shape:	 (1, 1, 3, 832)
Inception_block output shape:	 (1, 1, 3, 832)
Inception_block output shape:	 (1, 1, 3, 1024)
MaxPooling2D output shape:	 (1, 1, 2, 1024)
GlobalAveragePooling2D output shape:	 (1, 1024)
Flatten output shape:	 (1, 1024)
Dropout output shape:	 (1, 1024)
Dense output shape:	 (1, 10)
